In [ ]:
# 1. Clone do repositório
!git clone https://github.com/borges32/bot-trade.git
%cd bot-trade/forex-rl-dqn
!ls -la

In [ ]:
# 2. Instalar dependências
!pip install -q -r requirements.txt
print("✅ Dependências instaladas")

In [ ]:
# 2. Instalar dependências (somente essenciais)
import sys

print("📦 Instalando pacotes essenciais...")

# Instalar apenas o necessário para treinamento
!pip install -q torch gymnasium numpy pandas scikit-learn pyyaml 2>/dev/null

# Verificar instalação
try:
    import torch
    import gymnasium
    import pandas as pd
    import yaml
    from sklearn.preprocessing import StandardScaler
    
    print(f"\n✅ PyTorch: {torch.__version__}")
    print(f"✅ Gymnasium: {gymnasium.__version__}")
    print(f"✅ Pandas: {pd.__version__}")
    print(f"✅ PyYAML: OK")
    print(f"✅ Scikit-learn: OK")
    print("\n🎯 Todas as dependências necessárias instaladas!")
    
except ImportError as e:
    print(f"❌ Erro ao importar: {e}")
    print("Execute novamente a célula.")

In [ ]:
# 3. Upload do arquivo de dados
from google.colab import files

print("📤 Faça upload do arquivo usdjpy_history_30m.csv")
uploaded = files.upload()

# Mover para pasta data/
import shutil
for filename in uploaded.keys():
    shutil.move(filename, f'data/{filename}')
    print(f"✅ {filename} movido para data/")

!ls -lh data/

In [ ]:
# 4. Verificar GPU disponível
import torch
print(f"GPU disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# 5. TREINAR - 30M (120k steps, ~2-3 horas)
!python -m src.rl.train \
  --data data/usdjpy_history_30m.csv \
  --config config_30m.yaml \
  --artifacts artifacts_30m

In [ ]:
# 6. Verificar resultados
!ls -lh artifacts_30m/
!tail -50 artifacts_30m/*.log 2>/dev/null || echo "No log found"

In [ ]:
# 7. Download dos artefatos treinados
from google.colab import files
import os

# Compactar artifacts
!zip -r artifacts_30m.zip artifacts_30m/

# Download
files.download('artifacts_30m.zip')
print("✅ Download iniciado!")

## 📊 Análise Rápida dos Resultados

In [ ]:
# Mostrar métricas finais
import re

# Procurar por linhas de avaliação
!grep -A 4 "Final Evaluation" artifacts_30m/*.log 2>/dev/null || echo "Training still running..."